# Data Integration with CHAKRA

## Import CHAKRA library

In [1]:
include("src/Chakra.jl")
using Main.Chakra

## Implement an interface for a data source S1

- Module S1 is CHAKRA data source.
- It uses the Chakra reference implementation.
- It defines an attribute A whose type is Int.
- Its data content is a simple hierarchy containing one constituent Id(1)=>{:A=>10}

In [2]:
module S1

using Main.Chakra

Chakra.@Reference Int

@DefineAttribute :A Int

__data__ = ins(Id(1),seta(:A,10,agg(Constituent)),emp(Hierarchy))

end

Main.S1

In [3]:
Chakra.isdatasource(S1)

true

- The type of S1's data is S1.Hierarchy.
- S1.Hierarchy is a subtype of Chakra.Hierarchy.

In [4]:
typeof(S1.__data__)

Main.S1.Hierarchy

In [5]:
S1.__data__ isa Chakra.Hierarchy

true

- S1 defines methods for the generic Chakra functions.

In [6]:
dom(S1)

1-element Vector{Main.S1.Id}:
 Main.S1.Id(1)

Similarly, `cts` takes a hierarchy and returns a list of Id-Constituent pairs. 

In [7]:
cts(S1)

1-element Vector{Main.S1.Id}:
 Main.S1.Id(1)

`fnd` takes and id and a hierarchy and returns the constituent bound to that id. 

In [8]:
c1 = fnd(S1.Id(1),S1)

Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])

If an identifier is not present in a hierarchy, `fnd` will return None. 

In [9]:
fnd(S1.Id(2),S1)

Main.Chakra.None()

We can also extract attribute values from constituents. The attributes defined on a Chakra representation can be listed: (TODO: implement a proper function for this.)

In [10]:
methods(S1.__attributes__)

# 3 methods for generic function "__attributes__":
[1] __attributes__(a::Symbol) in Main.S1 at /Users/nick/Chakra/src/Chakra.jl:384
[2] __attributes__(::Val{:A}) in Main.S1 at /Users/nick/Chakra/src/Chakra.jl:392
[3] __attributes__(::Val{a}) where a in Main.S1 at /Users/nick/Chakra/src/Chakra.jl:381

`geta` takes a symbol representing an attribute name and a constituent and returns the value 

In [11]:
geta(:A,c1)

10

`geta` can also be applied to an id and a hierarchy. This is defined as fnd(x) >=> geta(a). 

In [12]:
geta("Main.S1.A",S1.Id(1),S1)

10

If a constituent has no value for an attribute, or if the constituent does not exist, `geta` will return None.

In [13]:
geta("Main.S1.A",S1.Id(2),S1)

Main.Chakra.None()

Trying to get attributes which have not been defined throws an error. 

In [14]:
geta(:B,c1)

LoadError: Attribute B is not defined in module Main.S1.

In [15]:
geta("Main.S1.B",c1)

LoadError: Name Main.S1.B is not defined globally.

## Implement a second data source interface S2

- S2 is a second Chakra data source.
- Again, it uses the reference implementation.
- It defines an attribtue B whose type is B.
- Its data content is a simple hierarchy containing one constituent Id("Two")=>{:B=>B(99)}

In [16]:
module S2

using Main.Chakra

Chakra.@Reference String

struct B
    value::Int
end

@DefineAttribute(:B,B)

__data__ = ins(Id("Two"),seta(:B,B(99),agg(Id)),emp(Hierarchy))

end
Chakra.isdatasource(S2)

true

In [17]:
dom(S2)

1-element Vector{Main.S2.Id}:
 Main.S2.Id("Two")

In [18]:
c2 = fnd(S2.Id("Two"),S2)

Main.S2.Constituent(Dict{Symbol, Any}(:B => Main.S2.B(99)), Dict{Symbol, Any}(), Main.S2.Id[])

In [19]:
geta(:B,c2)

Main.S2.B(99)

## Linking data sources S1 and S2 using a third data source S3

Module S3 is a data source which which encapsulates S1 and S2.

In [20]:
module S3

using Main.Chakra

using Main.S1, Main.S2

Chakra.@Reference Symbol [S1,S2]

@DefineProperty(:TYPE,String)

__data__ = ins(Id(:a),setp(:TYPE,"BinaryAssociation",agg(ID_TYPES[S1.Id(1),S2.Id("Two")])),emp(Hierarchy))

end

Main.S3

- Hierarchies in S3 exist on top of the hierarchies from S1 and S2. 
- S3 hierarchies are implicitly unioned with the data content of S1 and S2

In [21]:
isemp(emp(S3.Hierarchy))

false

S3 hierarchy destructors will dispatch to S1.data and S2.data. Calling cts and dom on S3 hierarchies returns not only the constituents defined in S3 but also those defined in S1.data and S2.data. Similarly, calling fnd on S3 hierarchies will dispatch according to the type of the identifier passed. 

In [22]:
cts(S3)

3-element Vector{Union{Main.S1.Id, Main.S2.Id, Main.S3.Id}}:
 Main.S3.Id(:a)
 Main.S2.Id("Two")
 Main.S1.Id(1)

In [23]:
c3 = fnd(S3.Id(:a),S3)

Main.S3.Constituent(Dict{Symbol, Any}(), Dict{Symbol, Any}(:TYPE => "BinaryAssociation"), Union{Main.S1.Id, Main.S2.Id, Main.S3.Id}[Main.S1.Id(1), Main.S2.Id("Two")])

In [24]:
fnd(S1.Id(1),S3)

Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])

In [25]:
fnd(S2.Id("Two"),S3)

Main.S2.Constituent(Dict{Symbol, Any}(:B => Main.S2.B(99)), Dict{Symbol, Any}(), Main.S2.Id[])

Operations derived from the Chakra interface can be generically applied to data from S3, leaving dispatch to the compiler. 

In [26]:
ps = pts(c3)

2-element Vector{Union{Main.S1.Id, Main.S2.Id, Main.S3.Id}}:
 Main.S1.Id(1)
 Main.S2.Id("Two")

In [27]:
s = sequence(ps,S3)

2-element Vector{Any}:
 Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])
 Main.S2.Constituent(Dict{Symbol, Any}(:B => Main.S2.B(99)), Dict{Symbol, Any}(), Main.S2.Id[])

In [28]:
sequence(S3.Id(:a),S3)

2-element Vector{Any}:
 Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])
 Main.S2.Constituent(Dict{Symbol, Any}(:B => Main.S2.B(99)), Dict{Symbol, Any}(), Main.S2.Id[])

In [29]:
x = gethead(s)

Main.S1.Constituent(Dict{Symbol, Any}(:A => 10), Dict{Symbol, Any}(), Main.S1.Id[])

In [30]:
geta(:A,x)

10

In [31]:
Chakra.peek(S3.__data__)

Pair{Union{Main.S1.Id, Main.S2.Id, Main.S3.Id}, Union{Main.S1.Constituent, Main.S2.Constituent, Main.S3.Constituent}}(Main.S3.Id(:a), Main.S3.Constituent(Dict{Symbol, Any}(), Dict{Symbol, Any}(:TYPE => "BinaryAssociation"), Union{Main.S1.Id, Main.S2.Id, Main.S3.Id}[Main.S1.Id(1), Main.S2.Id("Two")]))